In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MachineLearningPractice/Colab-Notebooks/FarmKit/foodfile/미식큐레시피.csv')

In [3]:
df.shape

(629, 12)

In [4]:
df.isnull().values.any()

False

In [5]:
df.head()

,번호,식품명,주재료,조리법,추천태그,이미지태그,지도검색어,description,ingredient,recipe,source,original name
0,1,가오리회무침,"생선류, 식초, 고추가루",무침,"#한식, #요리, #저녁",#해산물,홍어,새콤달콤한 양념에 무쳐낸 가오리회무침입니다.,"가오리, 오이, 미나리, 무, 붉은고추, 막걸리(세척용), 소금(1)절임용, 양 념...",1. 가오리는 내장을 제거한 후 씻어서 1 X 1 X 5cm로 얇게 저며 썬다.2....,팜킷,가오리회무침
1,2,냉채족발,"족발, 식초, 채소류",무침,"#한식, #요리, #음식점, #배달, #저녁",#고기,족발,족발을 해파리냉채와 곁들여 톡 쏘는 겨자소스에 찍어먹는 냉채족발입니다.,"돼지족, 물(1), 소주(1), 양파, 대파, 소금, 마늘, 생강, 통후추, 삼계탕...",[끓이기]1. 돼지족을 손질한 후 찬물에 2시간 담가 핏물을 뺸다.2. 돼지족에 물...,팜킷,족발
2,3,닭갈비,"닭고기, 고추장, 채소류",볶음,"#한식, #요리, #음식점, #배달, #그룹",#탕찌개,닭갈비,춘천식 닭갈비를 집에서도 간편하게 즐길 수 있는 요리입니다.,"닭다리살, 양배추, 고구마, 가래떡, 깻잎, 대파, 양파, 새송이버섯, 팽이버섯, ...","닭다리살은 깨끗하게 씻어 물기를 제거한다. 양배추, 고구마, 가래떡, 깻잎, 대파,...",Clova AI,
3,4,닭발구이,"닭발, 고추장",후라이팬구이,"#한식, #요리, #배달, #저녁",#고기,닭발,콜라겐이 풍부한 닭발을 석쇠에 구워 만든 구이입니다.,"닭발(무뼈), 쌀뜨물, 밀가루, 굵은소금, 닭발 재우기, 배즙, 양파즙, 양념장, ...",1. 손질된 뼈 없는 닭발을 구입해 쌀뜨물을 붓고 밀가루와 굵은소금을 넣어 바락바락...,팜킷,닭발구이
4,5,닭발볶음,"닭고기, 고추장",볶음,"#한식, #요리, #배달, #저녁",#고기,닭발,콜라겐이 풍부한 닭발을 매콤하게 볶아낸 술안주입니다.,"무뼈닭발, 마늘, 양배추, 소주(삶는용), 식용유(볶음용), 닭발 밑간, 소금, 맛...",1. 닭발은 끓는물에 소주를 넣고 푹 삶아준다.2. 삶은 닭발은 찬물에 씻고 체에 ...,팜킷,닭발볶음


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix_description = tfidf.fit_transform(df['description'])
tfidf_matrix_ingre = tfidf.fit_transform(df['ingredient'])
tfidf_matrix_recipe = tfidf.fit_transform(df['recipe'])

In [7]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

cosine_sim_description = cosine_similarity(tfidf_matrix_description, tfidf_matrix_description)
cosine_sim_ingre = cosine_similarity(tfidf_matrix_ingre, tfidf_matrix_ingre)
cosine_sim_recipe = cosine_similarity(tfidf_matrix_recipe, tfidf_matrix_recipe)
distance_sim_description = euclidean_distances(tfidf_matrix_description, tfidf_matrix_description)
distance_sim_ingre = euclidean_distances(tfidf_matrix_ingre, tfidf_matrix_ingre)
distance_sim_recipe = euclidean_distances(tfidf_matrix_recipe, tfidf_matrix_recipe)

In [8]:
indices = pd.Series(df.index, index=df['식품명']).drop_duplicates()
# 음식명을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개의 음식 목록 반환
def get_recommendations(title, sim, reverse):
  
  # 음식명을 통해서 전체 데이터 기준 그 음식의 index 값을 얻기
  idx = indices[title]

  # 유사도 매트릭스 에서 idx 에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
  sim_scores = list(enumerate(sim[idx]))

  # 유사도 기준 정렬
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=reverse)    # 코사인 유사도는 True, 유클리디안 유사도는 False를 넣어줘야함

  # 자기 자신을 제외한 10개의 추천 음식을 슬라이싱
  sim_scores = sim_scores[1:11]

  # 추천 음식목록 10개의 인덱스 정보 추출
  movie_indices = [i[0] for i in sim_scores]

  # 인덱스 정보를 통해 음식명 추출
  return list(df['식품명'].iloc[movie_indices])

In [9]:
def to_dataframe (cos_description, euc_description, cos_ingre, euc_ingre, cos_recipe, euc_recipe):
    df = pd.DataFrame()
    df['cos description'] = cos_description
    df['euc description'] = euc_description
    df['cos ingre'] = cos_ingre
    df['euc ingre'] = euc_ingre
    df['cos recipe'] = cos_recipe
    df['euc recipe'] = euc_recipe
    return df

In [10]:
to_dataframe(get_recommendations('냉채족발', cosine_sim_description, True), get_recommendations('냉채족발', distance_sim_description, False), get_recommendations('냉채족발', cosine_sim_ingre, True), get_recommendations('냉채족발', distance_sim_ingre, False), get_recommendations('냉채족발', cosine_sim_recipe, True), get_recommendations('냉채족발', distance_sim_recipe, False))

,cos description,euc description,cos ingre,euc ingre,cos recipe,euc recipe
0,닭고기냉채,닭고기냉채,족발,족발,족발,족발
1,불족발,불족발,불족발,불족발,불족발,불족발
2,족발,족발,굴무국,굴무국,보쌈,보쌈
3,육회비빔밥,육회비빔밥,해물완자전,해물완자전,도가니탕,도가니탕
4,모듬회덮밥,모듬회덮밥,닭발볶음,닭발볶음,뼈해장국,뼈해장국
5,월남쌈,월남쌈,감자된장국,감자된장국,소고기쌀국수,소고기쌀국수
6,쇠곱창구이,쇠곱창구이,도가니탕,도가니탕,꼬리곰탕,꼬리곰탕
7,카레돈까스,카레돈까스,쇠고기메추리알장조림,쇠고기메추리알장조림,우거지곰탕,우거지곰탕
8,순두부찌개,순두부찌개,숯불닭갈비,숯불닭갈비,소머리국밥,소머리국밥
9,모둠보쌈,모둠보쌈,풋고추된장무침,풋고추된장무침,순두부육개장,순두부육개장


In [11]:
def ingre_similarity(title1, title2):
  count = 0
  list = []
  idx1 = indices[title1]
  idx2 = indices[title2]
  ingre1 = df.iloc[idx1]['ingredient'].split(', ')
  ingre2 = df.iloc[idx2]['ingredient'].split(', ')
  print('첫 번째 재료 :' , ingre1 , '\n' + '두 번째 재료 :' , ingre2)
  for i in range(len(ingre1)):
    for j in range(len(ingre2)):
      if (ingre1[i] == ingre2[j]):
        list.append(ingre1[i])
        count+=1
  print('겹치는 재료 : ', list)
  print('겹치는 재료의 개수 :' , count)
  return

In [12]:
ingre_similarity('냉채족발', '족발')

첫 번째 재료 : ['돼지족', '물(1)', '소주(1)', '양파', '대파', '소금', '마늘', '생강', '통후추', '삼계탕용한방재료', '된장', '커피', '소주(2)', '양 념', '간장', '물엿'] 
두 번째 재료 : ['돼지족', '물(1)', '소주(1)', '양파', '대파', '소금', '마늘', '생강', '통후추', '삼계탕용한방재료', '된장', '커피', '소주(2)', '양 념', '간장', '물엿']
겹치는 재료 :  ['돼지족', '물(1)', '소주(1)', '양파', '대파', '소금', '마늘', '생강', '통후추', '삼계탕용한방재료', '된장', '커피', '소주(2)', '양 념', '간장', '물엿']
겹치는 재료의 개수 : 16


In [13]:
ingre_similarity('냉채족발', '불족발')

첫 번째 재료 : ['돼지족', '물(1)', '소주(1)', '양파', '대파', '소금', '마늘', '생강', '통후추', '삼계탕용한방재료', '된장', '커피', '소주(2)', '양 념', '간장', '물엿'] 
두 번째 재료 : ['돼지족', '물(1)', '소주(1)', '양파', '대파', '소금', '마늘', '생강', '통후추', '삼계탕용한방재료', '된장', '커피', '소주(2)', '양 념', '간장', '물엿']
겹치는 재료 :  ['돼지족', '물(1)', '소주(1)', '양파', '대파', '소금', '마늘', '생강', '통후추', '삼계탕용한방재료', '된장', '커피', '소주(2)', '양 념', '간장', '물엿']
겹치는 재료의 개수 : 16


In [14]:
ingre_similarity('냉채족발', '굴무국')

첫 번째 재료 : ['돼지족', '물(1)', '소주(1)', '양파', '대파', '소금', '마늘', '생강', '통후추', '삼계탕용한방재료', '된장', '커피', '소주(2)', '양 념', '간장', '물엿'] 
두 번째 재료 : ['굴', '무', '대파', '건멸치', '다시마', '건고추', '들기름', '다진마늘', '소주', '새우젓', '후추']
겹치는 재료 :  ['대파']
겹치는 재료의 개수 : 1


In [15]:
ingre_similarity('냉채족발', '닭발볶음')

첫 번째 재료 : ['돼지족', '물(1)', '소주(1)', '양파', '대파', '소금', '마늘', '생강', '통후추', '삼계탕용한방재료', '된장', '커피', '소주(2)', '양 념', '간장', '물엿'] 
두 번째 재료 : ['무뼈닭발', '마늘', '양배추', '소주(삶는용)', '식용유(볶음용)', '닭발 밑간', '소금', '맛술', '후춧가루', '양념장', '맛술', '후춧가루', '고춧가루', '간장', '설탕', '올리고당', '청양고추', '생강즙']
겹치는 재료 :  ['소금', '마늘', '간장']
겹치는 재료의 개수 : 3


In [16]:
ingre_similarity('냉채족발', '해물완자전')

첫 번째 재료 : ['돼지족', '물(1)', '소주(1)', '양파', '대파', '소금', '마늘', '생강', '통후추', '삼계탕용한방재료', '된장', '커피', '소주(2)', '양 념', '간장', '물엿'] 
두 번째 재료 : ['오징어', '칵테일새우', '양파', '대파', '당근', '두부', '달걀', '녹말가루', '밀가루', '반죽 양념', '소금', '후춧가루', '소주', '참기름', '식용유(부침용)']
겹치는 재료 :  ['양파', '대파', '소금']
겹치는 재료의 개수 : 3


In [17]:
ingre_similarity('냉채족발', '숯불닭갈비')

첫 번째 재료 : ['돼지족', '물(1)', '소주(1)', '양파', '대파', '소금', '마늘', '생강', '통후추', '삼계탕용한방재료', '된장', '커피', '소주(2)', '양 념', '간장', '물엿'] 
두 번째 재료 : ['닭다리살', '우유', '카레가루', '허브솔트', '올리브유', '파슬리가루', '월계수잎', '레몬즙', '소주', '매운고추', '쪽파']
겹치는 재료 :  []
겹치는 재료의 개수 : 0


In [18]:
ingre_similarity('냉채족발', '감자된장국')

첫 번째 재료 : ['돼지족', '물(1)', '소주(1)', '양파', '대파', '소금', '마늘', '생강', '통후추', '삼계탕용한방재료', '된장', '커피', '소주(2)', '양 념', '간장', '물엿'] 
두 번째 재료 : ['감자', '마늘', '양파', '대파', '육 수', '물', '멸치(대)', '된장']
겹치는 재료 :  ['양파', '대파', '마늘', '된장']
겹치는 재료의 개수 : 4


In [19]:
to_dataframe(get_recommendations('닭발구이', cosine_sim_description, True), get_recommendations('닭발구이', distance_sim_description, False), get_recommendations('닭발구이', cosine_sim_ingre, True), get_recommendations('닭발구이', distance_sim_ingre, False), get_recommendations('닭발구이', cosine_sim_recipe, True), get_recommendations('닭발구이', distance_sim_recipe, False))

,cos description,euc description,cos ingre,euc ingre,cos recipe,euc recipe
0,닭발볶음,닭발볶음,닭발볶음,닭발볶음,오징어채조림,오징어채조림
1,쇠갈비구이,쇠갈비구이,무조림,무조림,쥐포조림,쥐포조림
2,족발,족발,오징어채무침,오징어채무침,북어채무침,북어채무침
3,닭구이,닭구이,오징어채조림,오징어채조림,문어고추장볶음,문어고추장볶음
4,키조개구이,키조개구이,무짠지,무짠지,멸치마늘쫑볶음,멸치마늘쫑볶음
5,멸치풋고추볶음,멸치풋고추볶음,멸치마늘쫑볶음,멸치마늘쫑볶음,오징어채무침,오징어채무침
6,꽁치구이,꽁치구이,양미리조림,양미리조림,풋고추된장무침,풋고추된장무침
7,삼치구이,삼치구이,문어고추장볶음,문어고추장볶음,오리불고기,오리불고기
8,식빵토스트,식빵토스트,쥐포조림,쥐포조림,다시마무침,다시마무침
9,참치머리구이,참치머리구이,잔멸치풋고추볶음,잔멸치풋고추볶음,깻잎찜,깻잎찜


In [20]:
ingre_similarity('닭발볶음', '무조림')

첫 번째 재료 : ['무뼈닭발', '마늘', '양배추', '소주(삶는용)', '식용유(볶음용)', '닭발 밑간', '소금', '맛술', '후춧가루', '양념장', '맛술', '후춧가루', '고춧가루', '간장', '설탕', '올리고당', '청양고추', '생강즙'] 
두 번째 재료 : ['무', '고춧가루', '양파즙', '간장', '국간장', '참기름', '육 수', '멸치', '다시마', '물']
겹치는 재료 :  ['고춧가루', '간장']
겹치는 재료의 개수 : 2


In [21]:
ingre_similarity('닭발볶음', '오징어채무침')

첫 번째 재료 : ['무뼈닭발', '마늘', '양배추', '소주(삶는용)', '식용유(볶음용)', '닭발 밑간', '소금', '맛술', '후춧가루', '양념장', '맛술', '후춧가루', '고춧가루', '간장', '설탕', '올리고당', '청양고추', '생강즙'] 
두 번째 재료 : ['오징어(포)', '양념장', '간장 ', '고추장', '물엿', '통깨', '참기름', '양 념', '소금']
겹치는 재료 :  ['소금', '양념장']
겹치는 재료의 개수 : 2


In [22]:
ingre_similarity('닭발볶음', '오징어채조림')

첫 번째 재료 : ['무뼈닭발', '마늘', '양배추', '소주(삶는용)', '식용유(볶음용)', '닭발 밑간', '소금', '맛술', '후춧가루', '양념장', '맛술', '후춧가루', '고춧가루', '간장', '설탕', '올리고당', '청양고추', '생강즙'] 
두 번째 재료 : ['오징어(포)', '양념장', '물엿', '고추장', '간장', '설탕', '마늘', '깨소금', '참기름', '물']
겹치는 재료 :  ['마늘', '양념장', '간장', '설탕']
겹치는 재료의 개수 : 4


**지금 까지 본 것의 결론 : 겹치는 재료의 개수와 코사인 / 유클리디안 유사도는 상관관계가 없다... ?**

In [23]:
!pip install transformers
import torch
from transformers import AutoModel, AutoTokenizer

MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.2 MB/s 
     |████████████████████████████████| 596 kB 46.4 MB/s 
     |████████████████████████████████| 6.6 MB 72.1 MB/s 
     |████████████████████████████████| 101 kB 9.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
# [CLS] token을 얻기 위한 함수
def get_cls_token(sent_A):
    model.eval()
    tokenized_sent = tokenizer(
            sent_A,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=128
    )
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(    # **tokenized_sent
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

In [25]:
def cls_token_array(df, column) :
  dataset_cls_hidden = []
  for i in range(df.shape[0]):
      q_cls = get_cls_token(str(df.loc[i, column])) #column 예: '음식설명', '영문'
      dataset_cls_hidden.append(q_cls)
  dataset_cls_hidden = np.array(dataset_cls_hidden).squeeze(axis=1)
  #print(dataset_cls_hidden)   
  print(dataset_cls_hidden.shape) # column에 대한 [CLS] 토큰 벡터
  return dataset_cls_hidden

column = 'description'
dataset_cls_hidden = cls_token_array(df, column)

(629, 768)


In [26]:
def bert_food_recommendation(df, dataset_cls_hidden, column, food, similarity): 
  
  index = df[df['식품명'] == food].index[0]
  food_cls_hidden = get_cls_token(str(df.loc[index, column]))

  sim = similarity(food_cls_hidden, dataset_cls_hidden)
  #print(cos_sim)
  if similarity == cosine_similarity:
    top = np.argpartition(sim, -10)[:, -10:] # 상위 10개 인덱스 추출
  else :
    top = np.argpartition(sim, 10)[:, :10]

  return list(df.iloc[top[0]]['식품명'])

In [27]:
def to_dataframe2 (cos_description, euc_description, cos_ingre, euc_ingre, cos_recipe, euc_recipe):
    df = pd.DataFrame()
    df['cos description'] = cos_description
    df['euc description'] = euc_description
    df['cos ingre'] = cos_ingre
    df['euc ingre'] = euc_ingre
    df['cos recipe'] = cos_recipe
    df['euc recipe'] = euc_recipe
    return df

In [28]:
to_dataframe2(bert_food_recommendation(df, dataset_cls_hidden, 'description', '닭발구이', cosine_similarity), bert_food_recommendation(df, dataset_cls_hidden, 'description', '닭발구이', euclidean_distances), bert_food_recommendation(df, dataset_cls_hidden, 'ingredient', '닭발구이', cosine_similarity), bert_food_recommendation(df, dataset_cls_hidden, 'ingredient', '닭발구이', euclidean_distances), bert_food_recommendation(df, dataset_cls_hidden, 'recipe', '닭발구이', cosine_similarity), bert_food_recommendation(df, dataset_cls_hidden, 'recipe', '닭발구이', euclidean_distances))

,cos description,euc description,cos ingre,euc ingre,cos recipe,euc recipe
0,햄치즈샌드위치,꽈배기,쇠고기사태찜,김치볶음,육개장,물만두
1,닭발구이,후라이드치킨,참치볶음밥,오징어젓,단호박샐러드,감자크로켓
2,쇠고기된장찌개,닭구이,쇠고기볶음,양념갈치젓,만두육개장,해물죽
3,후라이드치킨,닭발구이,쇠고기볶음밥,참치볶음밥,김무침,오이지무침
4,꽈배기,쇠고기된장찌개,닭가슴살피망볶음,쇠고기사태찜,해물죽,돼지고기장조림
5,햄김치찌개,돼지고기찌개,낙지닭갈비,뼈찜,오이지무침,단호박샐러드
6,닭구이,우거지곰탕,양념갈치젓,낙지닭갈비,간장게장,김무침
7,우거지곰탕,햄김치찌개,오징어젓,문어고추장볶음,감자크로켓,만두육개장
8,돼지고기찌개,더덕장아찌,김치볶음,닭구이,물만두,육개장
9,간장양념닭다리튀김,두부된장찌개,뼈찜,닭가슴살피망볶음,돼지고기장조림,간장게장
